In [1]:
from json import load
from typing import Optional
from my_keys import MufexKeys
from quantfreedom.exchanges import Mufex
from quantfreedom.helpers.helper_funcs import round_size_by_tick_step
from pydantic import BaseModel, field_validator, model_validator, ConfigDict

%load_ext autoreload
%autoreload 2

In [2]:
class TradeInfo(BaseModel):
    asset_size: float
    asset_tick_step: int
    entry_price: Optional[float] = None
    entry_type: str
    exchange: str
    long_short: str
    sl_price: Optional[float] = None
    symbol: str
    tp_price: Optional[float] = None

    @field_validator("long_short")
    @classmethod
    def check_long_short(cls, value: str):
        if value not in ["long", "short"]:
            raise ValueError('long_short must be either "long" or "short"')
        return value

    @model_validator(mode="after")
    def model_checker(self):
        self.asset_size = round_size_by_tick_step(self.asset_size, self.asset_tick_step)
        self.symbol = self.symbol.upper()

        if self.entry_type == "limit":
            if self.entry_price == None:
                raise ValueError("entry price must be provided for limit orders")
            elif self.sl_price or self.tp_price:
                raise ValueError("stop loss and take profit must not be provided for limit orders")

        elif self.entry_type == "market":
            if self.entry_price:
                raise ValueError("entry price must not be provided for market orders")
            if self.tp_price and self.sl_price:
                if self.tp_price <= self.sl_price and self.long_short == "long":
                    raise ValueError("take profit price must be higher than stop loss price")
                elif self.tp_price >= self.sl_price and self.long_short == "short":
                    raise ValueError("take profit price must be lower than stop loss price")
        else:
            raise ValueError("entry type must be either 'limit' or 'market'")
        return self

    model_config = ConfigDict(
        # frozen=True,
        str_to_lower=True,
    )

In [6]:
trades_list: list[TradeInfo] = []

with open("data.json") as f:
    data = load(f)
    for trade in data:
        trade_info_single = TradeInfo(**trade)
        trades_list.append(trade_info_single)
print(trades_list[0])
trade_info = trades_list[-1]
trade_info

asset_size=0.001 asset_tick_step=3 entry_price=None entry_type='market' exchange='mufex' long_short='long' sl_price=69500.0 symbol='BTCUSDT' tp_price=71000.0


TradeInfo(asset_size=0.004, asset_tick_step=3, entry_price=None, entry_type='market', exchange='mufex', long_short='long', sl_price=69300.0, symbol='BTCUSDT', tp_price=70300.0)

In [4]:
mufex_exchange = Mufex(
    api_key=MufexKeys.mainnet_testacc_api_key,
    secret_key=MufexKeys.mainnet_testacc_secret_key,
    use_testnet=False,
)

In [7]:
mufex_exchange.create_long_hedge_mode_entry_market_order(
    symbol=trade_info.symbol,
    asset_size=trade_info.asset_size,
)
mufex_exchange.create_long_hedge_mode_sl_order(
    symbol=trade_info.symbol,
    asset_size=trade_info.asset_size,
    sl_price=trade_info.sl_price,
)
mufex_exchange.create_long_hedge_mode_tp_limit_order(
    symbol=trade_info.symbol,
    asset_size=trade_info.asset_size,
    tp_price=trade_info.tp_price,
)

'305cb520-63d8-4bfa-ace5-33dc7d056c2a'